In [1]:
using Pkg
Pkg.activate("..")
using ElementalFunctionsTests

  Activating project at `~/Documents/Academia/taylor/ElementalFunctionsTests`


In [2]:
using ProgressMeter
using JSON
using Printf

In [3]:
JSON.lower(e::Error32) =Dict(:x=>@sprintf("%a", e.x), :err=>e.err)

In [4]:
const r = range(-2, 2, length=41)
errors = []

Any[]

In [5]:
for f in mpfrfunctions
    x = nextfloat(-Inf32)
    maxerror = Error32(0.0, 0.0)
    h = buckets(r)
    
    f_c = eval(:(@cfunction($f, Cfloat, (Cfloat,))))
    f_mpfr = mpfrfun(string(f))
    dom = domain(Symbol(f))
    prog = Progress(4278190078, showspeed=true, dt=2.0, desc=string(f))
    sum=0    # To store sum of stream 
    sumsq=0  # To store sum of square of stream 
    n=0   

    
    while(!isinf(x))
        next!(prog)
        if x < dom[1]
            x = nextfloat(x)
            continue
        elseif x > dom[2]
            break;
        end
        
        err = ulperror32(f_c, f_mpfr, x)
        add2bucket!(h, r, err)
        abserr = abs(err)
        n+=1
        sum+=abserr
        sumsq+=(abserr*abserr) 
        
        if abs(err)>abs(maxerror.err)
          maxerror = Error32(x, err)
        end
        x = nextfloat(x)
    end

    mean = sum/n 
    var = (sumsq/n) - (mean*mean) 
    
    finish!(prog)
      
    push!(errors, TestsResults(Symbol(f),r,h, maxerror, 0, mean, var))
    open("Float32Errors.json","w") do f
      write(f, JSON.json(errors))
    end
end

cos 100%|██████████████████████████████████| Time: 4:14:24 ( 3.57 μs/it)m
sin 100%|██████████████████████████████████| Time: 4:25:07 ( 3.72 μs/it)
tan 100%|██████████████████████████████████| Time: 4:44:01 ( 3.98 μs/it)
cospi 100%|████████████████████████████████| Time: 3:42:57 ( 3.13 μs/it)
sinpi 100%|████████████████████████████████| Time: 4:01:46 ( 3.39 μs/it)
acos 100%|█████████████████████████████████| Time: 2:37:32 ( 2.21 μs/it)
asin 100%|█████████████████████████████████| Time: 3:30:26 ( 2.95 μs/it)
atan 100%|█████████████████████████████████| Time: 4:32:14 ( 3.82 μs/it)
csc 100%|██████████████████████████████████| Time: 4:47:30 ( 4.03 μs/it)
sec 100%|██████████████████████████████████| Time: 4:15:41 ( 3.59 μs/it)
cot 100%|██████████████████████████████████| Time: 4:52:30 ( 4.10 μs/it)
cosh 100%|█████████████████████████████████| Time: 3:49:05 ( 3.21 μs/it)
sinh 100%|█████████████████████████████████| Time: 3:51:58 ( 3.25 μs/it)
tanh 100%|█████████████████████████████████| Time: